In [1]:
import os
import pickle
from scipy import signal
import libfmp.b
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from load_djembe_marker import *
from foot_module import onset_calculations, onset_extraction, onset_filtering, utils, onset_plot
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter, argrelmin, argrelmax
from pydub import AudioSegment
from scipy.signal import find_peaks
from pydub.generators import Triangle
import mir_eval
from compute_tempo import *
from dance_evaluation import *

In [ ]:
pkl_filelist = os.listdir(f"/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/Dataset_V2")

for filename in pkl_filelist:
    
    pickle_path = f'./output/motion_data_pkl/{filename.replace("_Dancers.csv", "_T.pkl")}'
        
    if os.path.isfile(pickle_path):
        with open(pickle_path, 'rb') as file:
            motion_data = pickle.load(file)
        print(f"Loaded {filename}")

    onset_filename = filename.replace("_Dancers.csv", "")
    danceanno_filename = filename

    drum_onsets_path = f"/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/drum_onsets/{onset_filename}.csv"
    dance_anno_path = f"/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/Dataset_V2/{danceanno_filename}"

    print(pickle_path)
    print(drum_onsets_path)
    print(dance_anno_path)

    feet_length = len(motion_data['position']['SEGMENT_RIGHT_FOOT']) # size (n, 3)
    feet_onsets_path = f"./logs/{filename.replace("_Dancers.csv", "_T")}/onset_info/{filename.replace("_Dancers.csv", "_T")}_both_feet_onsets.csv"
    feet_df = pd.read_csv(feet_onsets_path, usecols=[1])
    feet_frames = feet_df.values.flatten()

    feet_onsets = np.zeros(feet_length)
    feet_onsets[feet_frames] = 1
    feet_onsets = np.reshape(feet_onsets, (-1, 1))
    #######################################################
    drum_df = pd.read_csv(drum_onsets_path)
    column_b = drum_df["J2"].dropna()
    drum_onsets = column_b.to_numpy()

    anno_df = pd.read_csv(dance_anno_path)
    category_df = anno_df.groupby('mocap')
    
    try:
        category_df = category_df.get_group('gr')
    except KeyError:
        # print("Group 'in' not found, continuing without it.")
        continue
    
    category_df = category_df.reset_index(drop=True)
    category_df
    
    row = 0
    start_f = np.round(category_df.iloc[row, 6]*240).astype(int)
    end_f = np.round(category_df.iloc[row, 7]*240).astype(int)
    start_t = start_f/240
    end_t = end_f/240
    ########################################################
    mocap_fps = 240
    window_size = 240*10 #corresponding to 10 sec
    hop_size = 240*5
    tempi_range = np.arange(60,200,1)

    tempo_json_two_sensor = main_two_sensor_feet(feet_onsets, mocap_fps, window_size, hop_size, tempi_range)
    feet_sensor_onsets = tempo_json_two_sensor["feet_sensor_onsets"]
    ########################################################
    drum_ref = drum_onsets[(drum_onsets >= start_t) & (drum_onsets <= end_t)]
    half_beats = (drum_ref[:-1] + drum_ref[1:]) / 2
    drum_subref = np.sort(np.concatenate((drum_ref, half_beats)))
    # raw onsets
    onsets_frm = np.where(feet_sensor_onsets > 0)[0]     # Frames idx of onsets
    onsets_t = onsets_frm/240
    dance_onset = onsets_t[(onsets_t >= start_t) & (onsets_t <= end_t)]

    print("\n")
    # evaluate_dance_onsets(drum_ref, dance_onset, tolerance=0.1)
    results = evaluate_dance_onsets_with_half_beats(drum_ref, dance_onset, tolerance=0.1)
    print("\nEvaluation Results:")
    for key, value in results.items():
        print(f"{key}: {value}")
    print("-"*40)

# Also check if the unmatched dance onset is half of its previous and next dance onset and not only with drum


Loaded BKO_E1_D1_08_Suku_Dancers.csv
./output/motion_data_pkl/BKO_E1_D1_08_Suku_T.pkl
/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/drum_onsets/BKO_E1_D1_08_Suku.csv
/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/Dataset_V2/BKO_E1_D1_08_Suku_Dancers.csv
Loaded BKO_E3_D5_03_Wasulunka_Dancers.csv
./output/motion_data_pkl/BKO_E3_D5_03_Wasulunka_T.pkl
/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/drum_onsets/BKO_E3_D5_03_Wasulunka.csv
/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/Dataset_V2/BKO_E3_D5_03_Wasulunka_Dancers.csv
Computing tempograms...
Tempograms generated
Computing max method...
Global Bpm: 85.36105439019863
Computing weighted method...
Global Bpm: 110.73134328358209



Evaluation Results:
Total Onsets: 62
Total Primary Match: 46
Total Secondary Match: 16
Primary Matching Rate: 0.7419354838709677
Secondary Matching Rate: 0.25806451612903225
Overall Matching Rate: 1.0
Average Time Difference (Primary): 0.056007246456521304
Average Time Difference (Secondary): 0.0480

In [ ]:
drum_df = pd.read_csv(drum_onsets_path)
column_b = drum_df["J2"].dropna()
drum_onsets = column_b.to_numpy()

anno_df = pd.read_csv(dance_anno_path)
category_df = anno_df.groupby('mocap')
category_df = category_df.get_group('in')
category_df = category_df.reset_index(drop=True)
category_df

In [ ]:
row = 0
start_f = np.round(category_df.iloc[row, 6]*240).astype(int)
end_f = np.round(category_df.iloc[row, 7]*240).astype(int)
start_t = start_f/240
end_t = end_f/240

drum_ref = drum_onsets[(drum_onsets >= start_t) & (drum_onsets <= end_t)]
half_beats = (drum_ref[:-1] + drum_ref[1:]) / 2
drum_subref = np.sort(np.concatenate((drum_ref, half_beats)))
# raw onsets
onsets_frm = np.where(feet_sensor_onsets > 0)[0]     # Frames idx of onsets
onsets_t = onsets_frm/240
dance_onset = onsets_t[(onsets_t >= start_t) & (onsets_t <= end_t)]

evaluate_dance_onsets(drum_ref, dance_onset, tolerance=0.1)

results = evaluate_dance_onsets_with_half_beats(drum_ref, dance_onset, tolerance=0.1)

print("\nEvaluation Results:")
for key, value in results.items():
    print(f"{key}: {value}")

print("-"*40)

### From extracted feet onsets

In [ ]:
novelty_length = len(feet_onsets)
time_axis = np.arange(novelty_length)/240

mocap_fps = 240
window_size = 240*10 #corresponding to 10 sec
hop_size = 240*5
tempi_range = np.arange(60,200,1)

tempo_json_two_sensor = main_two_sensor_feet(feet_onsets, mocap_fps, window_size, hop_size, tempi_range)
feet_sensor_onsets = tempo_json_two_sensor["feet_sensor_onsets"]
tempogram_ab = tempo_json_two_sensor["tempogram_ab"]
tempogram_raw = tempo_json_two_sensor["tempogram_raw"]
time_axis_seconds = tempo_json_two_sensor["time_axis_seconds"]
tempo_axis_bpm = tempo_json_two_sensor["tempo_axis_bpm"]

tempo_data_maxmethod = tempo_json_two_sensor["tempo_data_maxmethod"]
tempo_data_weightedkernel = tempo_json_two_sensor["tempo_data_weightedkernel"]
# tempo_data_combinedtempogram = tempo_json_two_sensor["tempo_data_combinedtempogram"]

# Max method
Aestimated_beat_pulse = tempo_data_maxmethod["estimated_beat_pulse"]
Atempo_curve = tempo_data_maxmethod["tempo_curve"]
Atempo_curve_time_axis = tempo_data_maxmethod["tempo_curve_time_axis"]
Aglobal_tempo_bpm = tempo_data_maxmethod["global_tempo_bpm"]

# Weighted method
Bestimated_beat_pulse = tempo_data_weightedkernel["estimated_beat_pulse"]
Btempo_curve = tempo_data_weightedkernel["tempo_curve"]
Btempo_curve_time_axis = tempo_data_weightedkernel["tempo_curve_time_axis"]
Bglobal_tempo_bpm = tempo_data_weightedkernel["global_tempo_bpm"]


In [ ]:
# Per mode: drum onset and directional change onset plot

drum_ref = drum_onsets[(drum_onsets >= start_t) & (drum_onsets <= end_t)]
half_beats = (drum_ref[:-1] + drum_ref[1:]) / 2
drum_subref = np.sort(np.concatenate((drum_ref, half_beats)))
# raw onsets
onsets_frm = np.where(feet_sensor_onsets > 0)[0]     # Frames idx of onsets
onsets_t = onsets_frm/240
dance_onset = onsets_t[(onsets_t >= start_t) & (onsets_t <= end_t)]

# estimated beats
time = np.arange(novelty_length) / mocap_fps
peaks, _ = signal.find_peaks(Bestimated_beat_pulse)  # , prominence=0.02
peaks_t = time[peaks]
beat_estimated = peaks_t[(peaks_t >= start_t) & (peaks_t <= end_t)]

print("ref max ioi:", max(np.diff(drum_ref)))
print("ref min ioi:", min(np.diff(drum_ref)))
print("ref average ioi:", np.average(np.diff(drum_ref)))
# Plot figure
plt.figure(figsize=(40, 6), dpi=300)
# plt.vlines(x= drum_ref, ymin=0.0, ymax=1, color='r', linestyle='dotted', linewidth=0.9)
plt.vlines(x= drum_ref, ymin=0.0, ymax=1, color='b', linewidth=1.5,)
plt.vlines(x= dance_onset, ymin=0.0, ymax=1, color='r', linewidth=0.9,)
plt.xlabel('Time (seconds)')
plt.title(f'{file_name}')
plt.grid(True)
plt.show()

In [ ]:
evaluate_dance_onsets(drum_ref, dance_onset, tolerance=0.1)

results = evaluate_dance_onsets_with_half_beats(drum_ref, dance_onset, tolerance=0.1)

print("\nEvaluation Results:")
for key, value in results.items():
    print(f"{key}: {value}")

print("-"*40)

In [ ]:
plt.figure(figsize=(40, 6), dpi=200)
plt.plot(100*Aestimated_beat_pulse[start_f:end_f], linewidth=1, color = 'b')
plt.plot(Atempo_curve[start_f:end_f], linewidth=1, color = 'r')
# plt.plot(Atempo_curve_time_axis, Atempo_curve, linewidth=0.9, color = 'red')
# plt.vlines(x=np.array([start_f, end_f]), ymin=0.0, ymax=100, color='g', linewidth=1)

plt.xlabel('Time (seconds)')
plt.ylabel('Tempo (BPM)')
plt.title(f'Start:{start_t} End:{end_t}')
plt.grid(True)
plt.show()

In [ ]:
# Tempogram

fig, axs = plt.subplots(1, 1, figsize=(5,5), dpi=100)
cax1 = axs.pcolormesh(time_axis_seconds, tempo_axis_bpm, tempogram_ab[0], shading='auto', cmap='magma')
axs.set_title('X-axis')
axs.set_xlabel('Time [s]')
axs.set_ylabel('Tempo [BPM]')
plt.colorbar(cax1, ax=axs, orientation='horizontal', label='Magnitude')
plt.show()

In [ ]:
# Evaluate using mir_eval
# A significant increase in Recall with a slight decrease in Precision 
# may indicate that dancers are indeed synchronizing with subdivisions.


half_beats = (drum_ref[:-1] + drum_ref[1:]) / 2
combined_ref = np.sort(np.concatenate((drum_ref, half_beats)))

# scores = mir_eval.beat.evaluate(drum_reference, beat_estimated)
print("drum reference | ")
precision, recall, f_measure = mir_eval.onset.f_measure(drum_ref, beat_estimated, window= 0.2)
print(f'Precision: {precision}, Recall: {recall}, F-Measure: {f_measure}')

print("combined half-beat | ")
precision, recall, f_measure = mir_eval.onset.f_measure(combined_ref, beat_estimated, window= 0.2)
print(f'Precision: {precision}, Recall: {recall}, F-Measure: {f_measure}')

# dance onsets are less than reference onsets and thus the evaluation gives poor results.
# if dance onsets are not present for reference onsets than it should not be penalized.

In [ ]:
onsets_val = drum_ref         # Onset values in seconds
window_size = 0.1  # 50ms window (±25ms)

# Plotting
plt.figure(figsize=(10, 2), dpi=300)
plt.vlines(x= dance_onset, ymin=0.0, ymax=1, color='b', linewidth=0.5,)
# Iterate through onsets and plot vertical lines and windows
for onset in onsets_val:
    window_start = onset - (window_size/2)  # Start of the window (25ms before)
    window_end = onset + (window_size/2)   # End of the window (25ms after)
    
    # Plot shaded window
    plt.axvspan(window_start, window_end, color='green', alpha=0.3)
    # Plot reference onset as a vertical line
    plt.axvline(onset, color='red', linestyle='--', linewidth=0.5)

# Customize the plot
plt.title("Visualization of Onsets with 50ms Windows")
plt.xlabel("Time (s)")
plt.xlim([min(onsets_val), max(onsets_val) + 1])  # Extend x-axis slightly beyond the last onset
plt.ylim([0, 1])  # Arbitrary y-axis for visualization
plt.yticks([])  # Hide y-axis ticks
plt.tight_layout()
plt.show()


In [ ]:

time = np.arange(novelty_length) / mocap_fps
peaks, properties = signal.find_peaks(Aestimated_beat_pulse)  # , prominence=0.02
beat_peaks_sec = time[peaks]

click_duration = 50  # milliseconds
click_freq = 1200  # Hz
file_name ="maraka"

# Generate a single click sound
click = Triangle(click_freq).to_audio_segment(duration=click_duration)

onset_times = beat_peaks_sec  # kept_onsets/240   beat_peaks_sec
dN = novelty_length
total_duration = (dN/240)*1000  #  in milliseconds

audio = AudioSegment.silent(duration=total_duration)
for onset in onset_times:
    position = int(onset * 1000)  # Convert onset time to milliseconds
    audio = audio.overlay(click, position=position)

# Export the audio with clicks to a file
audio.export(os.path.join("/itf-fi-ml/home/sagardu/extract_feet_onset", f"{file_name}_Both_Foot_weighted.wav"), format="wav")
# audio.export(os.path.join("/itf-fi-ml/home/sagardu/extract_feet_onset", f"{file_name}_Bothhand_dir.wav"), format="wav")